## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [2]:
#Load the libraries 
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet


#Load up store_income_dat_task.csv
df = pd.read_csv("store_income_data_task.csv", index_col = 0, keep_default_na = False) 
df.head()

/usr/local/Caskroom/miniforge/base/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,store_name,store_email,department,income,date_measured,country
id,,,,,,
1,"Cullen/Frost Bankers, Inc.",,Clothing,$54438554.24,4-2-2006,United States/
2,Nordson Corporation,,Tools,$41744177.01,4-1-2006,Britain
3,"Stag Industrial, Inc.",,Beauty,$36152340.34,12-9-2003,United States
4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
5,Mercantile Bank Corporation,,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [3]:
countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")

#remove white spaces and make all the letters lower case 
df['country'] = df['country'].str.lower()
df['country'] = df['country'].str.strip()
df['country'] = df['country'].str.strip(".")
df['country'] = df['country'].str.strip("/")

#remove any emptry values 
df = df[df['country']!= ""]

#print the unique countries after modifications 
df['country']


There are 77 unique countries


id
1                  united states
2                        britain
3                  united states
4                        britain
5                 united kingdom
                  ...           
996        s. africasouth africa
997                united states
998     united states of america
999                      england
1000              united kingdom
Name: country, Length: 914, dtype: object

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [4]:
#create a function that will help replace any repeats within the country names 
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    #get a list of unique strings
    strings = df[column].unique()

    #get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    #only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    #get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    #replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match

    #print done, to identify that the function has been carried out 
    print("done!")

replace_matches_in_column(df= df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df= df, column='country', string_to_match="uk")
replace_matches_in_column(df= df, column='country', string_to_match="united states")
replace_matches_in_column(df= df, column='country', string_to_match="united states of america")
replace_matches_in_column(df= df, column='country', string_to_match="s. africasouth africa")

#replace any names that represnt the same country with one consistent name choice, to avoid any repeats 
df.replace('uk', 'united kingdom', inplace=True)
df.replace('u.k.', 'united kingdom', inplace=True)
df.replace('u.k', 'united kingdom', inplace=True)
df.replace('britain', 'united kingdom', inplace=True)
df.replace('england', 'united kingdom', inplace=True)
df.replace('united states of america', 'united states', inplace=True)
df.replace('america', 'united states', inplace=True)
df.replace('sa', 'south africa', inplace=True)
df.replace('s.a.', 'south africa', inplace=True)
df.replace('s. africasouth africa', 'south africa', inplace=True)
df.replace('s.a', 'south africa', inplace=True)

#print out the unique countries left 
countries = df['country'].unique()

print(f"There are {len(countries)} unique countries")
countries


done!
done!
done!
done!
done!
There are 3 unique countries


array(['united states', 'united kingdom', 'south africa'], dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [7]:
#import the datetime library 
import datetime 

#create a specific format upon obtaining the today's date
today = "{dt.day}-{dt.month}-{dt.year}".format(dt = datetime.datetime.now())

#identify the "date_measured" data from the csv document 
date_measured = df['date_measured']

#identify both dates as a "date" format 
date_obj_1 = pd.to_datetime(today)
date_obj_2 = pd.to_datetime(date_measured, format = "%d-%m-%Y")

#subtract the dates from each other to get the "days_ago" column 
df["days_ago"] = date_obj_1 - date_obj_2
df["days_ago"]





id
1       6273 days
2       6304 days
3       7149 days
4       6180 days
5      18340 days
          ...    
996     6393 days
997    11799 days
998     5097 days
999     4469 days
1000    4147 days
Name: days_ago, Length: 914, dtype: timedelta64[ns]